In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Crypto_1.db')
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import timedelta

/Users/amos/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
symbols = pd.read_sql("""SELECT name FROM sqlite_schema WHERE type='table'""",engine)

In [3]:
symbols

,name
0,BTCUSDT
1,ETHUSDT


In [4]:
class SmaCross(Strategy):
    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        price = self.data.Close
        if crossover(self.sma1, self.sma2):
            sl = price - price * 0.03
            tp = price + price * 0.04
            self.buy(sl = sl, tp = tp)

In [5]:
def resampleOHLC(df, interval):
    df = df.resample(interval).agg({
    'Open':'first',
    'High':'max',
    'Low':'min',
    'Close':'last'})
    return df

In [6]:
def BT(interval):

    returns = []

    for symbol in symbols.name:
        qry = f"""SELECT * FROM '{symbol}' 
        WHERE date < '{pd.to_datetime('today') - timedelta(days=30)}'""" 
        df = pd.read_sql(qry,engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, SmaCross, cash=100000, commission=0.0015, exclusive_orders=True)
        output = bt.run()
        returns.append(output['Return [%]'])

    frame = pd.DataFrame(returns, index=symbols.name, columns=['ret'])
    top_5 = frame.nlargest(5, 'ret')
    return top_5, interval

In [11]:
top_5,interval = BT('15min')

In [9]:
def validate(top_5, interval):

    returns = []

    for symbol in top_5.index:
        qry = f"""SELECT * FROM '{symbol}' 
        WHERE date < '{pd.to_datetime('today') - timedelta(days=30)}'"""
        df = pd.read_sql(qry,engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, SmaCross, cash=100000, commission=0.0015, exclusive_orders=True)
        output = bt.run()
        returns.append(output['Return [%]'])

    outcome = pd.DataFrame(returns, index=top_5.index, columns=['ret'])
    return outcome

In [12]:
validate(top_5,interval)

,ret
name,
BTCUSDT,-14.690667
ETHUSDT,-19.656997


In [13]:
def run(interval):
    bt_results, interval = BT(interval)
    valid = validate(bt_results, interval)
    return valid.ret.mean()

In [14]:
run('15min')

-17.1738323600001